In [3]:
#Импортируем библиотеки

import requests
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt

#Зададим переменные для парсинга курса валют

start_date = '2023-06-01'
end_date = '2023-06-30'
base = 'BTC'
symbols = 'RUB'
format = 'CSV'

#Создадим переменную *response* и запишем в нее ответ сервера. Сохраним результат в формате *.csv.

response = requests.get('https://api.exchangerate.host/timeseries?',
                            params={'base': base,
                                    'start_date': start_date,
                                    'end_date': end_date,
                                    'symbols': symbols,
                                    'format': format
                            })

with open('./csv_files/exchange_june2023.csv', 'wb') as f:
    f.write(response.content)

#Прочитаем полученный файл exchange_june2023.csv средствами Pandas для замены разделителя десятичных знаков с "," на ".", чтобы данные корректно загрузились в базу данных.

df = pd.read_csv('./csv_files/exchange_june2023.csv', decimal=',', index_col=False)
df = pd.DataFrame(df)
df

#Снова сохраним таблицу в файл *.csv.

df.to_csv('./csv_files/exchange_june2023.csv', index=False)

#Создадим подключение к базе Postgres, которую мы развернули в docker-compose:

conn = psycopg2.connect(database='exchange',
                        user='postgres',
                        password='password',
                        host='localhost',
                        port=5432)
cur = conn.cursor()

#Создадим в базе таблицу exchange_june2023.

cur.execute("""
CREATE TABLE exchange_june2023(
    date DATE PRIMARY KEY,
    exchange_id VARCHAR,
    exchange_rate DECIMAL,
    base_exchange VARCHAR,
    start_date DATE,
    end_date DATE
)
""")

#Заполним таблицу данными из файла exchange_june2023.csv

cur.execute("""COPY exchange_june2023
FROM '/docker-entrypoint-initdb.d/csv_files/exchange_june2023.csv'
DELIMITER ','
CSV HEADER;""")
conn.commit()

#Посмотрим, как загрузились данные. Для этого выведем первую строчку таблицы.

cur.execute("""select * from exchange_june2023;""")
print(cur.fetchone())
conn.commit()

cur.execute("""CREATE TABLE result_query AS (SELECT * FROM (SELECT "date" date_max_rate, MAX(exchange_rate) max_val_rate
				FROM exchange_june2023
				WHERE exchange_rate = (SELECT MAX(exchange_rate) FROM exchange_june2023)
				GROUP BY "date") max_rate
CROSS JOIN
	(SELECT "date" date_min_rate, MIN(exchange_rate) min_val_rate
	FROM exchange_june2023
	WHERE exchange_rate = (SELECT MIN(exchange_rate) FROM exchange_june2023)
	GROUP BY "date") min_rate
CROSS JOIN
	(SELECT AVG(exchange_rate) avg_rate FROM exchange_june2023) avg_rate
CROSS JOIN
	(SELECT exchange_rate last_date_rate
						FROM exchange_june2023
						WHERE "date" = '2023-06-30') last_date_rate)
;""")
conn.commit()

#Выведем новую таблицу "result_query" на печать.

cur.execute("""select * from result_query;""")
print(cur.fetchall())
conn.commit()
conn.close ()

#[(datetime.date(2023, 6, 30), Decimal('2693520.583333'), datetime.date(2023, 6, 5), Decimal('2060826.642857'), Decimal('2312089.860954800000'), Decimal('2693520.583333'))]

#Вот так наша таблица выглядит в DBeaver'e.

plt.figure(figsize=(15, 30))
plt.axis('off')
image = plt.imread('result_query.jpg')
plt.imshow(image)
plt.show()

ModuleNotFoundError: No module named 'psycopg2'